In [ ]:
import streamlit as st
import pandas as pd
from MARKETING_CREATIVE_AISQL import snowbooks_extras

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
--this can be any stage with your images in it (S3 bucket, Azure blob, etc) with directory enabled
create or replace stage my_image_stage
DIRECTORY = (ENABLE = TRUE)
ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

In [ ]:
#upload the files from this example notebook to the stage.  The images can live in cloud storage (S3, Azure Blob, etc)
put_results = session.file.put(
    local_file_name="MARKETING_CREATIVE_AISQL/images/*",
    stage_location="@MY_IMAGE_STAGE",
    overwrite=True,
    auto_compress=False
)

for r in put_results:
    print(r.source, r.target, r.status)

In [ ]:
ALTER STAGE MY_IMAGE_STAGE REFRESH;

In [ ]:
--create a table to hold reference to images (image file location and approval being the key ones)

create or replace TABLE CREATIVE_IMAGE_REVIEW (
	APPROVED_BY VARCHAR,
    APPROVED_TIME TIMESTAMP,
    STATUS VARCHAR,
    IMG_FILE FILE,
	LAST_MODIFIED TIMESTAMP_TZ(3),
	MD5 VARCHAR,
	FILE_URL VARCHAR
);

In [ ]:
---this statement be run ad hoc in the streamlit app or part of a pipeline. It would surface new images for the AISQL to flag
insert into creative_image_review 
select null, null, null, to_file(file_url) img_file, LAST_MODIFIED, MD5, FILE_URL from directory(@MY_IMAGE_STAGE);

In [ ]:
select img_file, file_url, last_modified from creative_image_review

In [ ]:
--test out our prompt and images
SELECT
    img_file,
    file_url,
    'Needs Review - Alcohol' AS STATUS
FROM creative_image_review
  WHERE AI_FILTER('This image contains alcohol',img_file) 


In [ ]:
--the AISQL could be put into a pipeline, but we can make it more flexible using streamlit and allow live updates to the AI_FILTER criteria with a SQL like below
UPDATE creative_image_review
   SET status         = 'Needs Review - Alcohol',
       last_modified  = CURRENT_TIMESTAMP()      -- keep audit data fresh (optional)
 WHERE AI_FILTER('This image contains alcohol', img_file);

